In [1]:
import torch
from torch.autograd import Variable
import PIL.Image as Image
import torchvision
from torchvision import datasets, transforms
from deeplabv3.model.deeplabv3 import DeepLabV3
import numpy as np
import cv2
#from deeplabv3.utils.utils import label_img_to_color, crop_to_sidewalk
import pickle

DEEPLAB_PRETRAINED_PATH = '/Users/Devin/Documents/ml/bostonhacks2018/deeplabv3/pretrained_models/'
BASE_PATH = '/Users/Devin/Documents/ml/bostonhacks2018/'

In [2]:
model_id = '1'
project_dir = '/Users/Devin/Documents/ml/bostonhacks2018'
deeplab_model = DeepLabV3(model_id, project_dir)

pretrained resnet, 18


In [3]:
# Apply pretrained deeplab weights
deeplab_model.load_state_dict(
    torch.load(
        DEEPLAB_PRETRAINED_PATH + 'model_13_2_2_2_epoch_580.pth', 
        map_location='cpu'
    )
)


In [4]:
TEST_IMAGES_PATH = '/Users/Devin/Documents/ml/bostonhacks2018/test-images'

transform = transforms.Compose([
    transforms.ToTensor(),
])

testset = torchvision.datasets.ImageFolder(TEST_IMAGES_PATH, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=2, shuffle=False, num_workers=2)

In [ ]:
# Make predictions
preds_list = []
imgs_list = []
for loaded in testloader:
    imgs = Variable(loaded[0])
    preds_list.append(deeplab_model(imgs))
    imgs_list.append(imgs)

In [84]:
def crop_img_to_sidewalk(label_img, img):
    non_side_color = [255, 255, 255] #transparent

    img_height, img_width = label_img.shape

    img_color = np.zeros((img_height, img_width, 3))
    for row in range(img_height):
        for col in range(img_width):
            label = label_img[row, col]
            img_rgb = img[row, col]

            if label == 1: # is sidewalk
                img_color[row, col] = img_rgb
            else:
                img_color[row, col] = np.array(non_side_color)

    return img_color

def white_to_transparent(img_path):
    img = Image.open(img_path)
    img = img.convert("RGBA")

    pixdata = img.load()

    width, height = img.size
    for y in range(height):
        for x in range(width):
            if pixdata[x, y] == (255, 255, 255, 255):
                pixdata[x, y] = (255, 255, 255, 0)

    img.save(img_path, "PNG")

def create_sidewalk_segment(outputs, imgs):
    outputs = outputs.data.cpu().numpy()
    pred_label_imgs = np.argmax(outputs, axis=1) # (shape: (batch_size, img_h, img_w))
    pred_label_imgs = pred_label_imgs.astype(np.uint8)

    for i in range(pred_label_imgs.shape[0]):
        pred_label_img = pred_label_imgs[i] # (shape: (img_h, img_w))
        #img_id = img_ids[i]
        img = imgs[i] # (shape: (3, img_h, img_w))

        img = img.data.cpu().numpy()
        img = np.transpose(img, (1, 2, 0)) # (shape: (img_h, img_w, 3))
        #img = img*np.array([0.229, 0.224, 0.225])
        #img = img + np.array([0.485, 0.456, 0.406])
        img = img*255.0
        img = img.astype(np.uint8)

        pred_label_img_color = crop_img_to_sidewalk(pred_label_img, img)

        overlayed_img = pred_label_img_color #0.35*img + 0.65*
        overlayed_img = overlayed_img.astype(np.uint8)
            
        img_name = BASE_PATH + str(i) + "_overlayed.png"
        cv2.imwrite(img_name, overlayed_img) #i was img_id
        white_to_transparent(img_name)
        

In [86]:
create_sidewalk_segment(preds_list[0], imgs_list[0])